# Create UC Functions

In [ ]:
from insurance_operator_agents.crewai.crews import catalog, schema
%pip install -r ../requirements.txt --quiet
%restart_python

In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from shared.helper import *

in_workspace = is_running_in_databricks()
global_config = get_global_config(path='../global_config/.yaml')

if not in_workspace:
    from databricks.connect import DatabricksSession
    from dotenv import load_dotenv
    import os

    spark = DatabricksSession.builder.getOrCreate()
    load_dotenv('../.env')

    DATABRICKS_HOST = os.getenv('host')
    DATABRICKS_TOKEN = os.getenv('token')
else:
    DATABRICKS_HOST = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()

print(f"host: {DATABRICKS_HOST}")

INFO:helper:{'catalog': 'fins_genai', 'schema_agents': 'agents', 'schema_insurance_agents': 'insurance_agent', 'llm_endpoint_llama3p3_70B': 'databricks/databricks-meta-llama-3-3-70b-instruct', 'llm_endpoint_llama4_maverick': 'databricks/databricks-llama-4-maverick', 'llm_endpoint_sonnet_3p7': 'databricks/databricks-claude-3-7-sonnet'}
INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, serverless_compute_id, connection_timeout_seconds, token


host: adb-984752964297111.11.azuredatabricks.net


In [19]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = global_config['catalog']
SCHEMA = global_config['schema_insurance_agents']
print(f"Catalog: {CATALOG} | Schema: {SCHEMA}")

INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, serverless_compute_id, connection_timeout_seconds, token


Catalog: fins_genai | Schema: insurance_agent


# Create UC Tools

## `search_claims_details_by_policy_no()`

In [20]:
sql_function = \
f"""
create or replace function {CATALOG}.{SCHEMA}.search_claims_details_by_policy_no(
  input_policy_no STRING comment 'policy Number'
)
RETURNS table
COMMENT 'Returns claims details about a customer given policy number'
RETURN
SELECT *
FROM {CATALOG}.{SCHEMA}.claims_table
WHERE policy_no = input_policy_no
;
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
(spark.sql(f"select * from {CATALOG}.{SCHEMA}.search_claims_details_by_policy_no('102070455')")
 .show
 (truncate=False))

INFO:unitycatalog.ai.core.utils.retry_utils:Successfully re-acquired connection to a serverless instance.


+----------+------------------------------------+-------------------+---------+-------------------+---------------------+------------------+--------------------+-------------------------------------+---------------+----------+---------------------------+-------------------------+-------------+-------------+-----------------+------------------------+
|claim_date|claim_no                            |number_of_witnesses|policy_no|claim_amount_injury|claim_amount_property|claim_amount_total|claim_amount_vehicle|collision_number_of_vehicles_involved|collision_type |driver_age|driver_insured_relationship|driver_license_issue_date|incident_date|incident_hour|incident_severity|incident_type           |
+----------+------------------------------------+-------------------+---------+-------------------+---------------------+------------------+--------------------+-------------------------------------+---------------+----------+---------------------------+-------------------------+-------------+--

## `policy_doc_vector_search()`

In [24]:
sql_function = \
f"""
CREATE OR REPLACE FUNCTION {CATALOG}.{SCHEMA}.policy_doc_vector_search(
  query STRING comment 'The query string for searching insurance policy documentation.'
)
RETURNS TABLE
COMMENT 'Executes a search on insurance policy documentation to retrieve text documents most relevant to the input query.'
RETURN
SELECT
  chunked_text as page_content,
  map('doc_path', path, 'chunk_id', chunk_id) as metadata
FROM
  vector_search(
    index => '{CATALOG}.agents.policy_docs_chunked_files_vs_index',
    query => query,
    num_results => 3
  )
"""
sql_func_info = client.create_function(sql_function_body=sql_function)
(spark.sql(f"select * from {CATALOG}.{SCHEMA}.policy_doc_vector_search('car accident coverage')")
 .show())

INFO:unitycatalog.ai.core.utils.retry_utils:Successfully re-acquired connection to a serverless instance.


+--------------------+--------------------+
|        page_content|            metadata|
+--------------------+--------------------+
|MOTOR VEHICLE ACC...|{doc_path -> dbfs...|
|WHEN THIS COVERAG...|{doc_path -> dbfs...|
|P\nL\nE\nD\nO\nC\...|{doc_path -> dbfs...|
+--------------------+--------------------+

